In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import csv
import glob
from pathlib import Path
from datetime import datetime

# ================== CONFIGURATION ==================
# Check the "Input" section on the right sidebar to find your folder name
# It usually looks like: /kaggle/input/your-dataset-name/
INPUT_DIR = "/kaggle/input/dl-sprint-4-0-bengali-speaker-diarization-challenge/diarization/diarization/train/annotation" 

# We write outputs to the working directory
OUTPUT_RTTM = "/kaggle/working/annotations.rttm"
OUTPUT_LIST = "/kaggle/working/train.txt"

MIN_DURATION = 0.1  # Fix zero-length segments by adding 0.5s
# ===================================================

def time_to_seconds(time_str):
    if not time_str: return 0.0
    time_str = str(time_str).strip()
    try:
        pt = datetime.strptime(time_str, '%H:%M:%S')
        return pt.hour * 3600 + pt.minute * 60 + pt.second
    except ValueError:
        try:
            pt = datetime.strptime(time_str, '%M:%S')
            return pt.minute * 60 + pt.second
        except ValueError:
            return float(time_str)

# 1. Get all CSV files
csv_files = glob.glob(os.path.join(INPUT_DIR, "**/*.csv"), recursive=True)
print(f"Found {len(csv_files)} CSV files in {INPUT_DIR}")

unique_file_ids = []
stats = {"fixed_headers": 0, "fixed_durations": 0}

with open(OUTPUT_RTTM, 'w', encoding='utf-8') as rttm_out:
    for csv_path in csv_files:
        file_id = Path(csv_path).stem
        unique_file_ids.append(file_id)

        with open(csv_path, 'r', encoding='utf-8') as csv_in:
            reader = csv.reader(csv_in)
            for row in reader:
                if not row or len(row) < 3: continue
                
                # Auto-skip header
                if "start" in str(row[0]).lower():
                    stats["fixed_headers"] += 1
                    continue

                try:
                    start = time_to_seconds(row[0])
                    end = time_to_seconds(row[1])
                    speaker = row[2]
                    
                    # Fix negative/zero duration
                    if end <= start: end = start + MIN_DURATION
                    duration = end - start
                    if duration < MIN_DURATION: 
                        duration = MIN_DURATION
                        stats["fixed_durations"] += 1

                    line = f"SPEAKER {file_id} 1 {start:.3f} {duration:.3f} <NA> <NA> {speaker} <NA> <NA>\n"
                    rttm_out.write(line)
                except Exception as e:
                    pass # Skip unparseable lines silently

# 2. Write the file list
with open(OUTPUT_LIST, 'w', encoding='utf-8') as f:
    for fid in unique_file_ids:
        f.write(f"{fid}\n")

print(f"Success! Created {OUTPUT_RTTM}")
print(f"Fixed {stats['fixed_durations']} zero-duration segments.")

In [10]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("kaggle_token")


In [11]:
print(secret_value_0)

YOUR_HF_TOKEN_HERE


In [12]:
# Install dependencies
!pip install -q pyannote.audio torch torchvision torchaudio pytorch-lightning

# Login to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Create the database.yml file dynamically
config_content = f"""
Databases:
  MyDb: 
    - /kaggle/working/database.yml

Protocols:
  MyDb:
    SpeakerDiarization:
      MyProtocol:
        train:
          uri: {OUTPUT_LIST}
          annotation: {OUTPUT_RTTM}
          # This tells pyannote to look for WAV files in your Input folder
          database: {INPUT_DIR}/{{uri}}.wav 
"""

with open("/kaggle/working/database.yml", "w") as f:
    f.write(config_content)

print("database.yml created.")

In [ ]:
# 1. Force install compatible NumPy
!pip install "numpy<2.0" --upgrade --force-reinstall

# 2. Re-install pyannote to ensure it links to the old numpy
!pip install "pyannote.audio==3.1.1" 

# 3. Print instructions
print("------------------------------------------------------------------")
print("CRITICAL: You must now RESTART THE SESSION.")
print("Go to Menu: Run > Restart Session")
print("Do NOT run this cell again after restarting.")
print("------------------------------------------------------------------")

In [ ]:
# 1. Uninstall existing versions to clear the conflict
!pip uninstall -y pyannote.audio pyannote.core pyannote.database pyannote.metrics pyannote.pipeline torch-audiomentations torchmetrics
!pip uninstall -y numpy scipy

# 2. Reinstall compatible versions
# We pin pyannote.audio to 3.1.1 (Stable) and numpy to 1.x (Compatible)
!pip install "numpy<2.0" "scipy<1.13" 
!pip install "pyannote.audio==3.1.1"
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [1]:
# Install pyannote.audio normally. 
# We will fix the NumPy 2.0 issues in the next cell using a patch.
!pip install pyannote.audio

In [2]:
import numpy as np
import os

# ==========================================
#        THE NUMPY 2.0 FIX (PATCH)
# ==========================================
# 1. Restore np.NaN (removed in NumPy 2.0)
if not hasattr(np, "NaN"):
    np.NaN = np.nan

# 2. Patch 'scipy.signal.hamming' if missing (common in older audio libs)
try:
    import scipy.signal
    if not hasattr(scipy.signal, "hamming"):
        scipy.signal.hamming = scipy.signal.windows.hamming
except ImportError:
    pass

print(f"NumPy Version: {np.__version__}")
print("Patch applied successfully. Loading libraries...")

# ==========================================
#            NORMAL IMPORTS
# ==========================================
from pyannote.database import registry
from pyannote.audio import Model
from pyannote.audio.tasks import Segmentation
from pytorch_lightning import Trainer

print("SUCCESS: Libraries loaded without error.")

NumPy Version: 2.4.2
Patch applied successfully. Loading libraries...
'MyDb.SpeakerDiarization.MyProtocol' found in /kaggle/working/database.yml does not define the 'scope' of speaker labels (file, database, or global). Setting it to 'file'.


/usr/local/lib/python3.12/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.12/dist-packages/torchaudio/_internal/module_utils.py:71: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  return func(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/pyannote/audio/tasks/segmentation/mixins.py:37: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the

SUCCESS: Libraries loaded without error.


In [8]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("kaggle_token")


In [14]:
import os
os.environ["TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD"] = "1"

In [23]:
%%writefile /kaggle/working/database.yml
Protocols:
  MyDb:
    SpeakerDiarization:
      MyProtocol:
        scope: file
        train:
          uri: /kaggle/working/train.txt
          annotation: /kaggle/working/annotations.rttm
        

Overwriting /kaggle/working/database.yml


In [24]:
from pyannote.database import registry
from pyannote.audio.tasks import Segmentation

# 1. Load the database config (Ensure database.yml is clean as discussed before)
# It should strictly contain paths to .txt (lists) and .rttm (labels), NOT audio.
registry.load_database("/kaggle/working/database.yml")

# 2. DEFINE THE CORRECT PATH
# We map the file IDs to your specific input folder
AUDIO_FOLDER = "/kaggle/input/dl-sprint-4-0-bengali-speaker-diarization-challenge/diarization/diarization/train/audio"

def find_audio_path(file):
    # Pyannote passes a dictionary 'file' with a 'uri' key (the filename)
    # We construct the full path: FOLDER + FILENAME + .wav
    return f"{AUDIO_FOLDER}/{file['uri']}.wav" 

preprocessors = {
    "audio": find_audio_path
}

# 3. Load the protocol with the preprocessor
protocol = registry.get_protocol(
    "MyDb.SpeakerDiarization.MyProtocol", 
    preprocessors=preprocessors
)

print(f"Protocol loaded. Audio source set to: {AUDIO_FOLDER}")

# 4. Verify it works
# We try to grab the first file. If this fails, the file ID in train.txt 
# doesn't match the filename in the folder.
try:
    first_file = next(protocol.train())
    print("------------------------------------------------")
    print("SUCCESS! System found the first file.")
    print(f"ID: {first_file['uri']}")
    print(f"Path: {first_file['audio']}")
    print("------------------------------------------------")
except Exception as e:
    print("------------------------------------------------")
    print("STILL FAILING. Error details:")
    print(e)
    print(f"Please check if a file named '{file['uri']}.wav' actually exists in {AUDIO_FOLDER}")

Protocol loaded. Audio source set to: /kaggle/input/dl-sprint-4-0-bengali-speaker-diarization-challenge/diarization/diarization/train/audio
------------------------------------------------
SUCCESS! System found the first file.
ID: train_006
Path: /kaggle/input/dl-sprint-4-0-bengali-speaker-diarization-challenge/diarization/diarization/train/audio/train_006.wav
------------------------------------------------


In [28]:
import numpy as np

# --- MONKEY PATCH FOR NUMPY 2.0 ---
# Pyannote expects 'np.string_' which was removed in NumPy 2.0.
# We map it to 'np.bytes_' (which is the new name for the same thing).
if not hasattr(np, 'string_'):
    np.string_ = np.bytes_
    print("✅ FIXED: np.string_ has been restored.")
else:
    print("✅ No fix needed: np.string_ already exists.")
# ----------------------------------

✅ FIXED: np.string_ has been restored.


In [31]:
import torch
import torchaudio
from pyannote.database import registry
from pyannote.audio import Model
from pyannote.audio.tasks import Segmentation
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pyannote.core import Segment, Timeline

# 1. Load your custom database
registry.load_database("/kaggle/working/database.yml")

# --- THE FIX STARTS HERE ---
def preprocess_file(file):
    # Fix 1: Set the correct audio path
    # We construct the full path to the input directory
    path = f"/kaggle/input/dl-sprint-4-0-bengali-speaker-diarization-challenge/diarization/diarization/train/audio/{file['uri']}.wav"
    file["audio"] = path
    
    # Fix 2: Create the 'annotated' region (Fixes the TypeError)
    # Since we don't have a .uem file, we tell Pyannote that the 
    # entire audio file is valid for training.
    # We use torchaudio to quickly get the file duration.
    info = torchaudio.info(path)
    duration = info.num_frames / info.sample_rate
    
    # Create a Timeline covering from 0 to end of file
    file["annotated"] = Timeline([Segment(0, duration)])
    
    return file

preprocessors = {
    "audio": preprocess_file,      # Apply the path fix
    "annotated": preprocess_file   # Apply the region fix
}
# --- THE FIX ENDS HERE ---

protocol = registry.get_protocol("MyDb.SpeakerDiarization.MyProtocol",
                                preprocessors=preprocessors)

# 2. Load the base model
pretrained = "pyannote/segmentation-3.0"
model = Model.from_pretrained(pretrained, use_auth_token=True)
model=model.to('cuda')

# 3. Configure the Task
task = Segmentation(
    protocol,
    duration=10.0,
    max_num_speakers=4,
    batch_size=16,
    num_workers=2,
    loss="bce",
    vad_loss="bce"
)

# 4. Bind task to model
model.task = task

# 5. Checkpointing
checkpoint_callback = ModelCheckpoint(
    monitor="loss/train", 
    mode="min",
    save_top_k=1, 
    dirpath="/kaggle/working/models",
    filename="best_segmentation"
)

# 6. Trainer
trainer = Trainer(
    accelerator="gpu",
    devices=1,
    max_epochs=10,
    callbacks=[checkpoint_callback],
    default_root_dir="/kaggle/working/models",
    limit_val_batches=0,
    num_sanity_val_steps=0
    
)

# 7. Start Training
print("Starting training...")
trainer.fit(model)

/usr/local/lib/python3.12/dist-packages/pyannote/database/registry.py:499: UserWarning: Replacing existing MyDb.SpeakerDiarization.MyProtocol protocol by the one defined in '/kaggle/working/database.yml'.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/lightning_fabric/utilities/cloud_io.py:73: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
/tmp/ipykernel_311/3254616904.py:24: UserWarning: torchaudio._backend.utils.info has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  info = torchaudio.info(path)
/usr/local/lib/python3.12/dist-packages/torchaudio/_b

Protocol MyDb.SpeakerDiarization.MyProtocol does not provide the path to audio files: adding an 'audio' preprocessor for you. See pyannote.database documentation on how to do that yourself.
Protocol MyDb.SpeakerDiarization.MyProtocol does not precompute the output of torchaudio.info(): adding a 'torchaudio.info' preprocessor for you to speed up dataloaders. See pyannote.database documentation on how to do that yourself.
Starting training...


/tmp/ipykernel_311/3254616904.py:24: UserWarning: torchaudio._backend.utils.info has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  info = torchaudio.info(path)
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:20: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  s = torchaudio.io.S

┏━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   ┃ Name              ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃       In sizes ┃                  Out sizes ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0 │ sincnet           │ SincNet          │ 42.6 K │ train │ 960 M │ [1, 1, 160000] │               [1, 60, 589] │
│ 1 │ lstm              │ LSTM             │  1.4 M │ train │     0 │   [1, 589, 60] │    [[1, 589, 256], [[8, 1, │
│   │                   │                  │        │       │       │                │        128], [8, 1, 128]]] │
│ 2 │ linear            │ ModuleList       │ 49.4 K │ train │     0 │              ? │                          ? │
│ 3 │ classifier        │ Linear           │    516 │ train │ 603 K │  [1, 589, 128] │                [1, 589, 4] │
│ 4 │ activation        │ Sigmoid          │      0 │ train │     0 │    [1, 589, 4] │                [1, 589, 4] │
│ 5 │ validation_metric │ MetricCollection │      0 │ train │     0 │              ? │                          ? │
└───┴───────────────────┴──────────────────┴────────┴───────┴───────┴────────────────┴────────────────────────────┘

Trainable params: 1.5 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.5 M                                                                                                
Total estimated model params size (MB): 5                                                                          
Modules in train mode: 27                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 1.0 B

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/data.py:123: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.


Output()

/usr/local/lib/python3.12/dist-packages/pyannote/audio/tasks/segmentation/mixins.py:64: UserWarning: torchaudio._backend.common.AudioMetaData has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  file["torchaudio.info"] = AudioMetaData(
/usr/local/lib/python3.12/dist-packages/pyannote/audio/tasks/segmentation/mixins.py:64: UserWarning: torchaudio._backend.common.AudioMetaData has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more info

In [36]:
import torch
import torch.nn as nn
from pyannote.audio import Model, Pipeline

# 1. Load the default model architecture (which expects 7 speakers by default)
print("⏳ Loading base model...")
model = Model.from_pretrained("pyannote/segmentation-3.0", use_auth_token=True)

# --- THE SURGICAL FIX ---
# The error tells us the mismatch is in 'classifier.weight'.
# Default shape: [7, 128] (7 speakers)
# Your checkpoint: [4, 128] (4 speakers)
# We simply overwrite the final layer to match your checkpoint.
print("🔧 Manually resizing model output layer from 7 to 4 speakers...")
model.classifier = nn.Linear(128, 4) 
# ------------------------

# 2. Now load your weights
best_model_path = "/kaggle/working/models/best_segmentation.ckpt"
# (If your variable 'checkpoint_callback' is still alive, use that instead)

print(f"⏳ Loading weights from: {best_model_path}")
try:
    checkpoint = torch.load(best_model_path, map_location="cpu")
    model.load_state_dict(checkpoint["state_dict"])
    model.to(torch.device("cuda"))
    model.eval()
    print("✅ SUCCESS! Model loaded with 4-speaker configuration.")
except FileNotFoundError:
    print("❌ ERROR: Could not find the model checkpoint file. Check the path.")
except RuntimeError as e:
    print(f"❌ FATAL: Still getting a mismatch? Error: {e}")

# 3. Setup the Pipeline with your fixed model
try:
    pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", 
                                        use_auth_token=True)
    pipeline.segmentation_model = model
    pipeline.to(torch.device("cuda"))
    print("🚀 Pipeline is ready for testing!")
except Exception as e:
    print(f"⚠️ Pipeline Error: {e}")

⏳ Loading base model...


/usr/local/lib/python3.12/dist-packages/lightning_fabric/utilities/cloud_io.py:73: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
/tmp/ipykernel_311/1713129813.py:24: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  checkpoint = torch.load(best_model_path, map_location="cpu")


🔧 Manually resizing model output layer from 7 to 4 speakers...
⏳ Loading weights from: /kaggle/working/models/best_segmentation.ckpt
✅ SUCCESS! Model loaded with 4-speaker configuration.


config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/pyannote/audio/pipelines/speaker_verification.py:43: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  backend = torchaudio.get_audio_backend()
/usr/local/lib/python3.12/dist-packages/torchaudio/_internal/module_utils.py:71: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  return func(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part 

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

🚀 Pipeline is ready for testing!


In [38]:
import torch
import yaml
from huggingface_hub import HfApi, login

# --- CONFIGURATION ---
HF_USERNAME = "Risalat"  # <--- REPLACE THIS
REPO_NAME = "bengali-diarization-pipeline"
REPO_ID = f"{HF_USERNAME}/{REPO_NAME}"
TOKEN ="YOUR_HF_TOKEN_HERE" # <--- PASTE YOUR WRITE TOKEN HERE (or login via terminal)

# 1. Login
login(token=TOKEN)

# 2. Save your fine-tuned model weights
# We save it as 'pytorch_model.bin' because that's standard for HF
print("💾 Saving model weights...")
torch.save(model.state_dict(), "pytorch_model.bin")

# 3. Create the 'config.yaml'
# This is the recipe for the pipeline. We base it on the standard 3.1 pipeline
# but point the 'segmentation' field to our local .bin file.
config = {
    "version": "3.1.0",
    "pipeline": {
        "name": "pyannote.audio.pipelines.SpeakerDiarization",
        "params": {
            "clustering": "AgglomerativeClustering",
            "embedding": "pyannote/wespeaker-voxceleb-resnet34-LM", # Standard 3.1 embedding
            "embedding_batch_size": 32,
            "embedding_exclude_overlap": True,
            "segmentation": "pytorch_model.bin",  # <--- POINTS TO YOUR MODEL IN THE REPO
            "segmentation_batch_size": 32
        }
    },
    "params": {
        "clustering": {
            "method": "centroid",
            "min_cluster_size": 12,
            "threshold": 0.704 # You can tune this threshold later!
        },
        "segmentation": {
            "min_duration_off": 0.0
        }
    }
}

print("📝 Writing config.yaml...")
with open("config.yaml", "w") as f:
    yaml.dump(config, f)

# 4. Upload to Hugging Face
print(f"🚀 Uploading to {REPO_ID}...")
api = HfApi()

try:
    # Create the repo (private by default to be safe, change to False to make public)
    api.create_repo(repo_id=REPO_ID, exist_ok=True, private=False)
    
    # Upload Model
    api.upload_file(
        path_or_fileobj="pytorch_model.bin",
        path_in_repo="pytorch_model.bin",
        repo_id=REPO_ID
    )
    
    # Upload Config
    api.upload_file(
        path_or_fileobj="config.yaml",
        path_in_repo="config.yaml",
        repo_id=REPO_ID
    )
    
    print("✅ Upload Complete!")
    print(f"👉 You can now load it with: Pipeline.from_pretrained('{REPO_ID}')")

except Exception as e:
    print(f"❌ Error during upload: {e}")

💾 Saving model weights...
📝 Writing config.yaml...
🚀 Uploading to Risalat/bengali-diarization-pipeline...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

✅ Upload Complete!
👉 You can now load it with: Pipeline.from_pretrained('Risalat/bengali-diarization-pipeline')


In [42]:
import numpy as np
# --- PATCH START: Fix numpy compatibility issues ---
if not hasattr(np, "NAN"):
    np.NAN = np.nan
# --- PATCH END ---

import os
import torch
import torch.nn as nn
import torchaudio
import pandas as pd
import json
from pathlib import Path
from tqdm import tqdm
from pyannote.audio import Pipeline, Model
from kaggle_secrets import UserSecretsClient
from huggingface_hub import hf_hub_download

# --- 1. CONFIGURATION ---
TEST_DIR = "/kaggle/input/dl-sprint-4-0-bengali-speaker-diarization-challenge/diarization/diarization/test/audio"
MERGE_THRESHOLD = 0.5 
REPO_ID = "Risalat/bengali-diarization-pipeline" # <--- MAKE SURE THIS IS CORRECT

# Setup Secrets
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("kaggle_token") 

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")

# --- 2. PIPELINE SETUP (The "Surgical" Load) ---
print("📥 Downloading your custom model from Hugging Face...")

try:
    # A. Download the weights file manually
    model_path = hf_hub_download(repo_id=REPO_ID, filename="pytorch_model.bin", token=HF_TOKEN)

    # B. Initialize the empty architecture (Defaults to 7 speakers)
    model = Model.from_pretrained("pyannote/segmentation-3.0", use_auth_token=HF_TOKEN)

    # C. RESIZE LAYER TO 4 SPEAKERS (Crucial Fix)
    print("🔧 Resizing model to match 4-speaker training...")
    model.classifier = nn.Linear(128, 4)

    # D. Load the weights
    state_dict = torch.load(model_path, map_location="cpu")
    model.load_state_dict(state_dict)
    model.to(device)
    
    # E. Insert into Pipeline
    pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=HF_TOKEN)
    pipeline.segmentation_model = model
    pipeline.to(device)
    
    print("✅ Custom Pipeline loaded successfully!")

except Exception as e:
    print(f"❌ Critical Error Loading Model: {e}")
    raise e

# --- 3. HELPER FUNCTION ---
def merge_speaker_segments(diar_list, threshold=0.5):
    if not diar_list: return []
    temp_list = sorted([seg.copy() for seg in diar_list], key=lambda x: x['start_time'])
    merged = []
    current = temp_list[0]
    for next_seg in temp_list[1:]:
        if (next_seg['speaker_id'] == current['speaker_id'] and 
            (next_seg['start_time'] - current['end_time']) <= threshold):
            current['end_time'] = max(current['end_time'], next_seg['end_time'])
        else:
            merged.append(current)
            current = next_seg
    merged.append(current)
    return merged

# --- 4. MAIN LOOP ---
audio_files = sorted([f for f in os.listdir(TEST_DIR) if f.endswith(('.mp3', '.wav'))])
submission_data = []

print(f"🚀 Processing {len(audio_files)} files...")

with torch.no_grad():
    for filename in tqdm(audio_files):
        file_path = os.path.join(TEST_DIR, filename)
        
        try:
            # Load Audio manually
            waveform, sample_rate = torchaudio.load(file_path)
            
            # Run Inference
            diarization = pipeline({"waveform": waveform, "sample_rate": sample_rate})
            
            # Extract Segments
            raw_segments = []
            for turn, _, speaker in diarization.itertracks(yield_label=True):
                raw_segments.append({
                    "start_time": round(float(turn.start), 2),
                    "end_time": round(float(turn.end), 2),
                    "speaker_id": speaker
                })
            
            # Merge close segments
            final_segments = merge_speaker_segments(raw_segments, threshold=MERGE_THRESHOLD)
            
            # Add to submission
            file_id = Path(filename).stem
            submission_data.append({
                "filename": file_id,
                "diarization": json.dumps(final_segments)
            })
            
            # Cleanup memory
            del waveform
            del diarization

        except Exception as e:
            print(f"⚠️ Error processing {filename}: {e}")
            # If it fails, add an empty entry to avoid submission alignment errors
            submission_data.append({
                "filename": Path(filename).stem,
                "diarization": "[]"
            })
        
        # Periodic GPU cleaning
        if len(submission_data) % 10 == 0:
            torch.cuda.empty_cache()

# --- 5. SAVE SUBMISSION ---
df = pd.DataFrame(submission_data)
df.to_csv("submission.csv", index=False)
print("\n🎉 submission.csv generated successfully!")

🚀 Using device: cuda
📥 Downloading your custom model from Hugging Face...


/usr/local/lib/python3.12/dist-packages/lightning_fabric/utilities/cloud_io.py:73: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
/tmp/ipykernel_311/323913668.py:47: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  state_dict = torch.load(model_path, map_location="cpu")


🔧 Resizing model to match 4-speaker training...
✅ Custom Pipeline loaded successfully!
🚀 Processing 14 files...


  0%|          | 0/14 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3


🎉 submission.csv generated successfully!


In [44]:
import pandas as pd
import json

# 1. Load your data (Assuming it's in a dataframe or CSV)
# Replace this with: df = pd.read_csv("your_submission_file.csv")
# data = {
#     "filename": ["test_010"],
#     "diarization": [
#         # This is the raw string from your input provided above
#         '[{"start_time": 13.34, "end_time": 15.29, "speaker_id": "SPEAKER_10"}, {"start_time": 14.00, "end_time": 16.00, "speaker_id": "SPEAKER_05"}]' 
#         # Note: I added a fake overlapping segment (SPEAKER_05) above to demonstrate the fix
#     ]
# }
df = pd.read_csv("submission.csv")

def resolve_overlaps_greedy(json_str):
    """
    Parses segment list and strictly enforces 'First Speaker Wins' logic.
    """
    try:
        segments = json.loads(json_str)
    except:
        return json_str # Handle empty or malformed rows

    # 1. Sort primarily by start_time
    segments.sort(key=lambda x: x['start_time'])

    resolved_segments = []
    last_end_time = 0.0

    for seg in segments:
        original_start = seg['start_time']
        original_end = seg['end_time']
        
        # Determine the effective start time
        # If the previous speaker is still talking, this speaker has to wait
        new_start = max(original_start, last_end_time)
        
        # Only keep the segment if it still has valid duration
        if new_start < original_end:
            # Update the segment with the trimmed start time
            seg['start_time'] = round(new_start, 2)
            resolved_segments.append(seg)
            
            # Update the global "busy until" timer
            # We use the original end because this speaker holds the floor until then
            last_end_time = original_end
        else:
            # This segment was completely engulfed by the previous speaker
            # It is removed entirely strictly following "First speaker ID assigned"
            pass

    return json.dumps(resolved_segments)

# 2. Apply the fix
df['diarization'] = df['diarization'].apply(resolve_overlaps_greedy)

# 3. Inspect the result (or save to CSV)
# You will see overlaps are gone, and sequential order is preserved.
print(df.iloc[0]['diarization'])

df.to_csv("submission_pyannote_bangla_trained.csv", index=False)

[{"start_time": 13.34, "end_time": 15.29, "speaker_id": "SPEAKER_10"}, {"start_time": 16.43, "end_time": 18.34, "speaker_id": "SPEAKER_10"}, {"start_time": 19.11, "end_time": 24.12, "speaker_id": "SPEAKER_10"}, {"start_time": 24.68, "end_time": 25.25, "speaker_id": "SPEAKER_10"}, {"start_time": 26.0, "end_time": 27.33, "speaker_id": "SPEAKER_10"}, {"start_time": 29.35, "end_time": 30.55, "speaker_id": "SPEAKER_10"}, {"start_time": 31.72, "end_time": 44.69, "speaker_id": "SPEAKER_10"}, {"start_time": 58.09, "end_time": 59.23, "speaker_id": "SPEAKER_10"}, {"start_time": 60.81, "end_time": 61.11, "speaker_id": "SPEAKER_10"}, {"start_time": 62.28, "end_time": 63.03, "speaker_id": "SPEAKER_10"}, {"start_time": 64.41, "end_time": 65.0, "speaker_id": "SPEAKER_10"}, {"start_time": 67.6, "end_time": 69.77, "speaker_id": "SPEAKER_10"}, {"start_time": 114.32, "end_time": 136.49, "speaker_id": "SPEAKER_16"}, {"start_time": 143.56, "end_time": 146.05, "speaker_id": "SPEAKER_10"}, {"start_time": 147